In [1]:
import pandas as pd
import numpy as np
import os

from thefuzz import fuzz #new fuzzywuzzy project https://github.com/seatgeek/thefuzz
import textdistance

# "Library"

In [2]:
def isNaN(value):
    if type(value)==str:
        return (value.upper() == 'NAN')
    else:
        return np.isnan(value)
    
def areEquivalentValues(val1, val2, equivalences):
    return (val1==val2 or (val1,val2) in equivalences) or ((val2,val1) in equivalences)

In [3]:
def compatibleRanges(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'] == 'all'
    
    #for debugging
    val1 = val2 = 'NAN'
    
    for field in options['fields'].split(' '):        
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        else:
            compatible &= abs(int(row1[field]) - int(row2[field])) <= int(options['parameter'])
            val1 = int(row1[field])
            val2 = int(row2[field])
            
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}, range:{int(options['parameter'])}")
    
    return compatible

def compatibleCategory(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'].upper() == 'ALL'
    equivalences = None
    
    #for debugging
    val1 = val2 = 'NAN'
    
    if not isNaN(options['parameter']):
        equivalencesDF = pd.read_csv(f"{configPath}/{options['parameter']}")
        equivalences = list(equivalencesDF.itertuples(index=False, name=None))
        # WARNING, there may be a better way to do this other than creating tuples but we should have a scheme
        # to follow, in order to grab the columns without having to specify their names
    
    for field in options['fields'].split(' '):
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        elif equivalences:
            compatible &= areEquivalentValues(row1[field], row2[field], equivalences)
            val1 = row1[field]
            val2 = row2[field]
    
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}")
    
    return compatible

In [4]:
def areCompatibles(row1, row2, verbose=False):
    compatible = True
        
    for variable, options in comparisonSettings.items():    
        if options['consider'].lower() == 'yes':
            if verbose:
                print(f"{variable} compatible?")
            varCompatible = True

            if options['type'] == 'range':
                varCompatible = compatibleRanges(row1, row2, options, verbose)
            elif options['type'] == 'categorical':
                varCompatible = compatibleCategory(row1, row2, options, verbose)

            compatible &= varCompatible
            if verbose:
                print(f"\tresult: {varCompatible}")
    if verbose:
        print(f"Candidates? {compatible} \n\n")
    return compatible

In [5]:
def areCandidates(row1, row2, verbose=False):
    candidates = True

    for scheme, key1, key2, threshold in schemesConfig.itertuples(index=False):
        #check for valid scheme and keys
        if scheme not in schemes:
            if verbose:
                print(f"scheme not used: {scheme}")
            continue
        if verbose:
            print(f"scheme {scheme}")
        if key1 not in base1.columns:
            print(f"invalid key on database 1 {key1}, skipping")
            continue
        if key2 not in base2.columns:
            print(f"invalid key on database 2 {key2}, skipping")
            continue

        value1 = row1[key1]
        value2 = row2[key2]
        
        information = {}
        
        for algorithmName, algorithm in comparisonAlgorithms.items():
            result = algorithm(value1, value2)
            candidates &= (result >= threshold)

            information[f"{algorithmName}_{scheme}"] = result
    return candidates, pd.Series(information)

# Config

In [6]:
#config files
configPath = os.path.join(os.getcwd(), 'config')
dictionariesPath = os.path.join(os.getcwd(), 'dicts')

In [7]:
#input
inputPath = os.path.join(os.getcwd(), 'example/instances')

base1 = pd.read_csv(f"{inputPath}/inst1/base_inst1.csv")
base2 = pd.read_csv(f"{inputPath}/inst2/base_inst2.csv")

#output
outputPath = os.path.join(inputPath, 'matches')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
    
outputFileName = 'candidateList'

In [8]:
variableFields = pd.read_csv(f"{configPath}/variableFields.csv").set_index('variable')
compatibleData = pd.read_csv(f"{configPath}/compatible_data.csv").set_index('variable')

comparisonSettings = compatibleData.join(variableFields).to_dict(orient='index')

compatibleData.join(variableFields) #just to visualize

,type,parameter,consider,na.action,fields
variable,,,,,
Case.ID,Case.ID,NaN,no,NaN,Case.ID
cod,cod,NaN,no,NaN,cod
nationality,categorical,natEquivalences.csv,yes,all,Nat_PROC Nat_2_PROC
sex,categorical,NaN,yes,all,Sex_PROC
age,range,6,yes,all,Age_PROC Age_2_PROC


dtype('float64')

In [13]:
schemes = pd.read_csv(f"{configPath}/select_schemes.csv", names=['schemes'])['schemes'].to_list() 
print(f"schemes {schemes}")
#TODO add option for "fast scheme"

schemesConfig = pd.read_csv(f"{configPath}/info_scheme.csv")
schemesConfig.rename(columns = {'listA.column': 'key1', 'listB.column': 'key2'}, inplace=True)
if schemesConfig.dtypes['threshold'] == str:
    schemesConfig.threshold = schemesConfig.threshold.str.replace(',', '.').astype(float)
schemesConfig

schemes ['A_A', 'F_F']


,scheme,key1,key2,threshold
0,A_A,Name_A,Name_A,0.3
1,F_F,Father_name_A,Father_name_A,0.0
2,M_M,Mother_name_A,Mother_name_A,0.0
3,AD_AD,Adress_A,Adress_A,0.0
4,Ph_Ph,Phone_1_PROC Phone_2_PROC Phone_3_PROC,Phone_1_PROC Phone_2_PROC Phone_3_PROC,0.0


In [ ]:
comparisonAlgorithms = {
    'Levenshtein': textdistance.levenshtein.normalized_similarity,
    'DL': textdistance.damerau_levenshtein.normalized_similarity,
    'fuzzy_ratio' : fuzz.ratio
} #TODO add the rest/more

# Script

## Find compatible rows

In [ ]:
compatiblesDF = base1.apply(lambda row1: base2.apply(lambda row2: areCompatibles(row1, row2), axis=1), axis=1)
i1, i2 = (compatiblesDF.values).nonzero()
compatibles = list(zip(i1, i2))

compatibles

## Use the schemes configured on the compatible pairs

In [ ]:
candidatesList = []

for i1, i2 in compatibles:
    row1 = base1.iloc[i1]
    row2 = base2.iloc[i2]
    
    candidates, information = areCandidates(row1, row2)

    if candidates:
        #rename columns as base1_ and base2_
        row1.set_axis(['base1_'+x for x in base1.columns], inplace=True)
        row2.set_axis(['base2_'+x for x in base2.columns], inplace=True)
        
        candidatesList.append(pd.concat([row1, row2, information]))

candidatesDF = pd.concat(candidatesList, axis=1).T

candidatesDF.to_csv(f"{outputPath}/candidatesList.csv", index=False)
candidatesDF

# Only for checking

In [ ]:
exBase1 = base1.iloc[4]
exBase2 = base2.iloc[12]

In [ ]:
exBase1

In [ ]:
exBase2

In [ ]:
#TEST

#Stringdist (R) Functions
    #Levenshtein distance
print("levenshtein normalized similarity", textdistance.levenshtein.normalized_similarity('test', 'tast'))
    
    #D-L 
        #The restricted Damerau-Levenshtein distance is not a true distance metric because it does not 
        #satisfy the triangle inequality. This makes it a poor choice for applications that involve evaluating 
        #the similarity of more than two strings, such as clustering.
print("damerau-levenshtein normalized", textdistance.damerau_levenshtein.normalized_similarity('test', 'tast'))

    #D-L-FULL ???
    
    #LongestCommonSubstring 
    #falta encontrar uno que haga tal cuál se quiere, sino se puede calcular, pero con cuidado
ex1 = 'Jonh Smit'
ex2 = 'John Smith'
substr = textdistance.lcsstr(ex1, ex2)
print(substr)
print("longest common substring", 1-len(substr)/(len(ex1)+len(ex2))) #TODO
    
    #GramX

#FuzzyWuzzyFunctions
    #Ratio
print("fuzz.ratio('test', 'tast')", fuzz.ratio('test', 'tast'))
    #partial_ratio
print("fuzz.partial_ratio('test', 'tast!')", fuzz.partial_ratio('test', 'tast!'))
print("fuzz.partial_ratio('test', 'test!')", fuzz.partial_ratio('test', 'test!'))
    #token_sort_ratio
print("fuzz.ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear')", fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))
print("fuzz.token_sort_ratio('fuzzy wuzzy was a bear', 'wuzzy fuzzy was a bear')", fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))
    #WRATIO == Set Ratio?
print("fuzz.token_sort_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear')", fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))
print("fuzz.token_set_ratio('fuzzy was a bear', 'fuzzy fuzzy was a bear')", fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear"))

In [ ]:
print(base1.shape)
base1

In [ ]:
print(base2.shape)
base2.head(14)